In [1]:
import tensorflow as tf

import numpy as np
import os
import time

import urllib.request


In [2]:
urls=["https://www.gutenberg.org/files/7205/7205-0.txt",
      "http://www.gutenberg.org/cache/epub/7204/pg7204.txt",
      "http://www.gutenberg.org/cache/epub/7202/pg7202.txt",
      "https://www.gutenberg.org/files/60360/60360-0.txt"]


In [3]:
text = ""
for url in urls:
    with urllib.request.urlopen(url) as f:
        text += f.read().decode('utf-8')[900:-13000]

In [4]:
len(text)

1868534

In [9]:
# Take a look at the first 250 characters in text
print(text[:250])

in Buch für Alle und Keinen


von Friedrich Wilhelm Nietzsche



Inhaltsverzeichnis


 Erster Theil
 Zarathustra’s Vorrede
 Die Reden Zarathustra’s
 Von den drei Verwandlungen
 Von den Lehrstühlen der Tugend
 Von den Hinterweltlern
 V


In [10]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

144 unique characters


In [11]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [12]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  ...
}


In [13]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'in Buch für A' ---- characters mapped to int ---- > [ 68  73   2  31  80  62  67   2  65 108  77   2  30]


In [14]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

i
n
 
B
u


In [15]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'in Buch für Alle und Keinen\r\n\r\n\r\nvon Friedrich Wilhelm Nietzsche\r\n\r\n\r\n\r\nInhaltsverzeichnis\r\n\r\n\r\n Erst'
'er Theil\r\n Zarathustra’s Vorrede\r\n Die Reden Zarathustra’s\r\n Von den drei Verwandlungen\r\n Von den Leh'
'rstühlen der Tugend\r\n Von den Hinterweltlern\r\n Von den Verächtern des Leibes\r\n Von den Freuden- und L'
'eidenschaften\r\n Vom bleichen Verbrecher\r\n Vom Lesen und Schreiben\r\n Vom Baum am Berge\r\n Von den Predi'
'gern des Todes\r\n Vom Krieg und Kriegsvolke\r\n Vom neuen Götzen\r\n Von den Fliegen des Marktes\r\n Von der'


In [16]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [17]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'in Buch für Alle und Keinen\r\n\r\n\r\nvon Friedrich Wilhelm Nietzsche\r\n\r\n\r\n\r\nInhaltsverzeichnis\r\n\r\n\r\n Ers'
Target data: 'n Buch für Alle und Keinen\r\n\r\n\r\nvon Friedrich Wilhelm Nietzsche\r\n\r\n\r\n\r\nInhaltsverzeichnis\r\n\r\n\r\n Erst'


In [18]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 68 ('i')
  expected output: 73 ('n')
Step    1
  input: 73 ('n')
  expected output: 2 (' ')
Step    2
  input: 2 (' ')
  expected output: 31 ('B')
Step    3
  input: 31 ('B')
  expected output: 80 ('u')
Step    4
  input: 80 ('u')
  expected output: 62 ('c')


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [22]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 144) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           36864     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 144)           147600    
Total params: 4,122,768
Trainable params: 4,122,768
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
sampled_indices

array([131, 133,  17, 132, 109,  70,  45, 110, 100,  72,  50,  71, 133,
        27,  20,   0,  27,  80, 100,  24,  38,  33, 122,  67,  73,  50,
       131,   6,  54,  31, 127, 101, 118, 118,  25, 129,  82,  29,  87,
        99,   5,  16, 118, 113,  18,  85,  23, 112,  74, 124,  43, 130,
        11,   1,  18,  41, 122,  73, 112,  39, 118,  52,  91,  47,  54,
         7, 143,  59,  97, 119,  10, 106,  22,  26, 127, 105,  36,  47,
        49,  10,  64,  17,   7, 114, 115,   8,  68,  53,  73,  24,  75,
        84, 131,   6, 136,  42, 128,  96,  46,  79])

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Werden taucht; die habende, welche in’s\r\nWollen und Verlangen _will_:—\r\n\r\n—die sich selber fliehende'

Next Char Predictions: 
 "ἴὲ2άœkPαêmUlὲ=5\n=uê9IDπhnUἴ&YBωëκκ:ἄw@~é#1κε3z8δoςNἐ,\r3LπnδJκWÜRY'„`çλ*ö7;ωôGRT*e2'ζη(iXn9pyἴ&όMἀäQt"


In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 144)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.9704895


In [29]:
model.compile(optimizer='adam', loss=loss)

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [31]:
EPOCHS=10

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
289/289 [==============================] - 16s 56ms/step - loss: 2.3640
Epoch 2/10
289/289 [==============================] - 16s 57ms/step - loss: 1.6918
Epoch 3/10
289/289 [==============================] - 17s 57ms/step - loss: 1.4766
Epoch 4/10
289/289 [==============================] - 17s 57ms/step - loss: 1.3797
Epoch 5/10
289/289 [==============================] - 17s 58ms/step - loss: 1.3185
Epoch 6/10
289/289 [==============================] - 17s 58ms/step - loss: 1.2725
Epoch 7/10
289/289 [==============================] - 17s 59ms/step - loss: 1.2342
Epoch 8/10
289/289 [==============================] - 17s 59ms/step - loss: 1.2002
Epoch 9/10
289/289 [==============================] - 17s 59ms/step - loss: 1.1690
Epoch 10/10
289/289 [==============================] - 17s 59ms/step - loss: 1.1395


In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            36864     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 144)            147600    
Total params: 4,122,768
Trainable params: 4,122,768
Non-trainable params: 0
_________________________________________________________________


In [38]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [39]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: nicht unheil, Flamme und
Vergleichung muss, sie sollten die Instinkte *wissen* -- daß du jeder Mangel an einer überwältigten
Schritt, ohne dass mir meine Seele in allen ungesetzte Zukünfte.


141.

Die *Moral der Leibe schneidet* -- -- Das Leben ist ungerecht gefragt: katig, unsäuciglich den Freunde,
den Krämern engeln, hob bis sie verbeiste: denne wird ihm Schopenhauerswürdiger: was Alles
warn und schliessen schon: die nannte
Zarathustra. Geben wir zu fordern hat. - "Letzteren
sind die Begierde nach Namen Mitte an sich ein nur auf Posen“. Wir haben nur
wünschen, um seiner Seele auf,
sie ist eine Traussigkeit. Die begreifliche Katon entellte *Herdenist*
nach Verbheichnist und einen Willen zur Achtung sein wiegel
wird, -- : daß es denkbo wild es diese Dipfernd
lieben, er siebliche Kraft allein gab mir auch sein und der
Macht rolltegnoss ein solches, das vorsichtet werden zu lassen, sondern leugnet sie
noch an Scham ist, oder ein Ander- nämlich diese Frage in Gespenster
